# 2-7 GCJの問題に挑戦してみよう(1)
### MinimumScalar Product（2008 Round1A A）

<img src="images/2-7/2023-04-22-17-41-43.png" width="800">

### 解説
ベクトルの要素を大きい順と小さい順に並べ替えて、掛け算して足すのみ

In [3]:
# 入力
v1 = [1, 2, 3, 4, 5]
v2 = [1, 0, 1, 0, 1]
n = len(v1)

v1.sort()
v2.sort(reverse=True)
temp = [v1_n * v2_n for v1_n, v2_n in zip(v1, v2)]
print(sum(temp))

6


### Crazy Rows(2009 Round2 A)

<img src="images/2-7/2023-04-22-17-54-28.png" width="800">

### 解説

つまりi行目のリストのx個目(x > i)の要素に1が含まれないように行を入れ替える必要がある

In [9]:
def resolve(M):
    MAX_N = 40

    N = len(M)
    a = [-1] * MAX_N  # a[i]はi行目の最後に現れる1の位置-1～N-1

    res = 0
    for i in range(N):
        a[i] = -1  # i行目に1がない場合は-1にしておく
        for j in range(N):
            if M[i][j] == 1:
                a[i] = j

    for i in range(N):
        pos = -1  # i行目に移動する行
        for j in range(i, N):
            if a[j] <= i:
                pos = j
                break

        # 実際にスワップする
        for j in range(pos, i, -1):
            a[j], a[j - 1] = a[j - 1], a[j]
            res += 1

    print(res)
    print(a)


MAX_ROUND = [[1, 1, 1, 0], [1, 1, 0, 0], [1, 1, 0, 0], [1, 0, 0, 0]]
resolve(MAX_ROUND)

4
[0, 1, 2, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


### Bribe the Prisoners (2009 Round 1C C)

<img src="images/2-7/2023-04-23-09-23-10.png" width="800">

<img src="images/2-7/2023-04-23-09-30-32.png" width="800">

<img src="images/2-7/2023-04-23-09-31-27.png" width="800">

### 解説

ある囚人を解放した際、連続している部分が2つに分かれて、その2つはもうお互いに関係しない、というところがポイント。

まずdp[i][j]には、開放する囚人リストaの中の、$a_i$の囚人と$a_j$の囚人の"間の開放するべき囚人"を全員開放するコスト(開放したくない囚人は含めない)の最小値を入れることとする。

<img src="images/2-7/2023-04-23-09-37-18.png" width="600">

上の図で☆の囚人を解放した場合、

- そのときに必要な金貨の数。
- 左側の部分(①)内での解放で必要な金貨の数。= d[i][☆]
- 右側の部分(②)内での解放で必要な金貨の数。= d[☆][j]

の3つの合計が必要な金貨の数となる。これを、最初に解放する囚人☆をすべて試して再帰的に計算すれば答えが求まる。

i~jの幅wが小さい順からdpを埋めていけば、再帰的に計算できる。

ここで、再帰的に計算する過程で計算対象となる連続した部分というのは、両端が端または解放される人であることがわかります。

したがって、計算対象となる連続部分は$O(Q^2)$個しかありません。よって、動的計画法により順番に計算していくことで、$0(Q^3)$で計算することができます。

※$dp[i][j]$はi人目の囚人からj人目の囚人まで(両端の囚人は含まない)の部分になったときに、その中の囚人をすべて解放するのに必要な金貨の最小枚数。

※端を簡単に扱うため、0人目の囚人を左端、Q+1人目の囚人を右端とする。したがって、$dp[0][Q+1]$が答え。

In [ ]:
def solve(prisoner_cnt, release_prisoner_cnt, virtual_release_prisoner_list):
    # 初期化
    MAX_Q = 100
    INF = float("inf")

    # 開放する囚人のリストを準備する, release_prisoner_cnt = 解説のQである。
    virtual_release_prisoner_list = [None] * (release_prisoner_cnt + 2)
    # このあとの操作を簡単にするため、開放する囚人リスト両端に、
    # 部屋番号0, prisoner_count + 1の仮想の囚人を追加する
    virtual_release_prisoner_list[0] = 0
    virtual_release_prisoner_list[release_prisoner_cnt + 1] = prisoner_cnt + 1
    # その間に実際に開放する囚人の番号(1スタート)を入力
    virtual_release_prisoner_list[1 : release_prisoner_cnt + 1] = release_prisoner_list

    # dp[i][j] := 囚人iと囚人jの間の囚人を解放するのに必要な金貨
    # dpを初期化
    # なぜMAX_Q + 2を使うのか?
    dp = [[None] * (release_prisoner_cnt + 2) for _ in range(release_prisoner_cnt + 1)]

    # i, jの幅wが小さい順にdpを埋めていく
    # まず幅w=1、つまり[i], [j] = [i + 1] のケース。間に開放すべき囚人はいないので必要な金貨は0で固定
    for i in range(release_prisoner_cnt + 1):
        dp[i][i + 1] = 0

    # w = 2以上のケース
    # つまりw = 2 から "release_prisoner_cnt + 1"までの整数について調べる
    for w in range(2, release_prisoner_cnt + 2):
        for i in range(release_prisoner_cnt + 2 - w):
            # dp[i][j] (i, jの"間"の開放リスト内の囚人を開放するコスト)を計算する
            j = i + w

            # まずどんな開放の順番であれ、最初の囚人開放にかかるコストは同じで、
            # i, jの間の開放しない囚人を含めたの囚人の人数 - 1(開放される囚人自身) = (j番目 - i番目 - 1) - 1のコストがかかる
            initial_release_cost = (
                virtual_release_prisoner_list[j] - virtual_release_prisoner_list[i] - 2
            )

            remaining_release_cost = INF

            # 最初に解放する囚人initial_release_prisonerがi ~ j - 1のどれかであるすべてのケースを試し
            # 残りの囚人を開放する最小のコストを探す
            # initial_release_prisonerを開放したあとの開放コストは、
            # dp[i][initial_release_prisoner] + dp[initial_release_prisoner][j]であり、
            # この2つはこの時点のwより小さいwのdpの計算ですでに求まっている
            for initial_release_prisoner in range(i + 1, j):
                remaining_release_cost = min(
                    remaining_release_cost,
                    dp[i][initial_release_prisoner] + dp[initial_release_prisoner][j],
                )

            dp[i][j] = initial_release_cost + remaining_release_cost

    # 答え
    print(dp[0][release_prisoner_cnt + 1])


# 入力
"""
prisoner_cnt = 8
release_prisoner_list = [3]
"""
prisoner_cnt = 20  # 囚人の数(部屋の数)
release_prisoner_list = [3, 6, 14]
release_prisoner_cnt = len(release_prisoner_list)  # 解放する囚人の数


solve(prisoner_cnt, release_prisoner_cnt, release_prisoner_list)

### Millionaire (2008 APAC local onsites C)

<img src="images/2-7/2023-04-30-12-28-08.png" width="800">

### 解説

各ラウンドにかけることができるお金は少数も選択できるので、無数の可能性があると思われるが、よく考えると有限個の場合だけ考えればいいことに気づく。

最後のラウンドMでは以下の3通りとなる

 - 持ち金X $\geqq$ $1,000,000の場合、賭けを終了すれば良い。確率1でお金が得られる
 - 持ち金X $\geqq$ $500,000の場合は全額賭ける。確率Pでお金が得られる
 - 持ち金X $< $ $500,000の場合はお金が得られる確率は0

最後から1回前のラウンドM-1では、以下の5通りになる

 - 持ち金X $\geqq$ \$1,000,000の場合、賭けを終了すれば良い。確率1でお金が得られる
 - 持ち金X $\geqq$ \$750,000の場合、\$250,000賭ける。2回チャンスがあり1回は負けても良い。確率$2P$でお金が得られる
 - 持ち金X $\geqq$ \$500,000の場合、全額賭ける。1回勝てば良い。負けは許されない。確率Pでお金が得られる
 - 持ち金X $\geqq$ \$250,000の場合は全額賭ける。2回勝つ必要があるので確率$ P^2 $でお金が得られる
 - 持ち金X $< $ $250,000の場合はお金が得られる確率は0

これをどんどんさかのぼっていくと、残りMラウンド(1回目のラウンド)の確率のパターンは、$2^M + 1$個分考えれば良い。

ここからdpを設計していく、dpに必要な要素数は行が2つ(i, i + 1)(メモリ節約のため)、と列(最大ケース数)$2^M+1$。



In [ ]:
def solve(MAX_ROUND, INITIAL_MONEY, P):
    GOAL = 1000000  # 目標金額
    MAX_M = 15
    n = 1 << MAX_ROUND  # 2^M
    # dpを初期化
    dp = [[0.0] * (n + 1) for _ in range(2)]

    preview = dp[0]  # 最終ラウンド後(残り0ラウンド)
    # 最後の要素($1Mもう持っている場合)は上がれる確率1
    # それ以外は0
    preview[n] = 1.0
    next = dp[1]  # 残り1ラウンド

    for round in range(MAX_ROUND):
        # 目標額GOALのn分割を1単位としたときの現時点の所持金をiとする
        # i = 0は所持金0,
        # i = iは(GOAL/n) * iの所持金,
        # i = nは所持金がGOALを達成している状態
        # ここで、
        # i = 所持金
        # jub = 賭けて意味のある(勝ったときに最低限GOALを超える)金額のうち、賭けられる最大の金額
        # つまり、jub = min(n - i, i)しか取れない。"n - i"はGOALまでに必要な残り金額
        # j = 試行時の賭金
        # i + j = 買った場合の所持金
        # i - j = 負けた場合の所持金
        for i in range(n + 1):
            jub = min(i, n - i)  # GOALまでの残り金額(n - 1)か、所持金のどちらか少ない方が最大の賭金
            t = 0.0
            # next[i]:1回前のラウンドで所持金iの状態から、
            # 勝つ場合: 確率Pでpreview[i + j]の状態に遷移。
            # 負ける場合: 確率1-Pでpreview[i - j]の状態に遷移。
            # なのでnext[i]からGOALを達成できる最大の確率はすべての賭金jのケースで試して一番大きい
            # t = "P * preview[i + j] + (1 - P) * preview[i - j]"を計算すれば良い
            for j in range(jub + 1):
                t = max(t, P * preview[i + j] + (1 - P) * preview[i - j])
            next[i] = t
        preview, next = next, preview  # previewを使いまわし

    i = INITIAL_MONEY // (GOAL / n)  # 最初の所持金をiになおす
    print(preview[i])


"""
M = 1
P = 0.5,
X= 500000
"""
MAX_ROUND = 3  # 最大ラウンド数
P = 0.75  # 勝つ確率
INITIAL_MONEY = 600000  # 始めに持っている金額

solve(MAX_ROUND, INITIAL_MONEY, P)